# 2- How to use Dynamic Masking 


> With Dynamic Masking, the database owner can hide personnal data for some users, 
> while other users are still allowed to read and write the authentic data.

## The Story


Paul has 2 employees : 

* Jack is operating the new sales application, he needs access to the real data. He is what the GPDR would call a **"data processor"**
* Pierre is a data analyst who runs statistic queries on the database. He should not have access to any personnal data. 

## How it works

![](./img/anon_dynamic.png)

## Objectives

In this section, we will learn:

* How to write simple masking rules
* The advantage and limitations of dynamic masking
* The concept of "Linkability" of a person


In [1]:
%reload_ext sql
%sql postgresql://paul@localhost/boutique
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

## The "company" table

In [2]:
%%capture
%%sql
DROP TABLE IF EXISTS supplier CASCADE;
DROP TABLE IF EXISTS company CASCADE;

CREATE TABLE company (
    id SERIAL PRIMARY KEY,
    name TEXT,
    vat_id TEXT UNIQUE
);

In [3]:
%%capture
%%sql
INSERT INTO company 
VALUES 
(952,'Shadrach', 'FR62684255667'),
(194,E'Johnny\'s Shoe Store','CHE670945644'),
(346,'Capitol Records','GB663829617823')
;

In [4]:
%%sql 
SELECT * FROM company;

id,name,vat_id
952,Shadrach,FR62684255667
194,Johnny's Shoe Store,CHE670945644
346,Capitol Records,GB663829617823


## The "supplier" table

In [5]:
%%capture
%%sql
CREATE TABLE supplier (
    id SERIAL PRIMARY KEY,
    fk_company_id INT REFERENCES company(id), 
    contact TEXT, 
    phone TEXT,
    job_title TEXT
);

In [6]:
%%capture 
%%sql 
INSERT INTO supplier 
VALUES 
(299,194,'Johnny Ryall','597-500-569','CEO'),
(157,346,'George Clinton', '131-002-530','Sales manager')
;

In [7]:
%%sql 
SELECT * FROM supplier;

id,fk_company_id,contact,phone,job_title
299,194,Johnny Ryall,597-500-569,CEO
157,346,George Clinton,131-002-530,Sales manager


## Activate the extension

In [8]:
%%capture
%%sql
CREATE EXTENSION IF NOT EXISTS anon CASCADE;
SELECT anon.init();

## Dynamic Masking

### Activate the masking engine

In [9]:
%%capture
%%sql
SELECT anon.start_dynamic_masking();

### Masking a role

In [10]:
%%capture
%%sql
SECURITY LABEL FOR anon ON ROLE pierre IS 'MASKED';
GRANT ALL ON SCHEMA public TO jack;
GRANT ALL ON ALL TABLES IN SCHEMA public TO jack;

--- 

Now connect as Pierre and try to read the supplier table :

In [11]:
%sql postgresql://pierre@localhost/boutique

In [12]:
%%sql
SELECT * FROM supplier;

id,fk_company_id,contact,phone,job_title
299,194,Johnny Ryall,597-500-569,CEO
157,346,George Clinton,131-002-530,Sales manager


For the moment, there is no masking rule so Pierre can see the original data in each table.

## Masking the supplier names

Connect as Paul and define a masking rule on the supplier table 

In [13]:
%sql postgresql://paul@localhost/boutique

In [14]:
%%capture
%%sql 
SECURITY LABEL FOR  anon ON COLUMN supplier.contact
IS 'MASKED WITH VALUE $$CONFIDENTIAL$$';

---

Now connect as Pierre and try to read the supplier table again:

In [15]:
%sql postgresql://pierre@localhost/boutique

In [16]:
%%sql
SELECT * FROM supplier;

id,fk_company_id,contact,phone,job_title
299,194,CONFIDENTIAL,597-500-569,CEO
157,346,CONFIDENTIAL,131-002-530,Sales manager


---

Now connect as Jack, try to read the real data

In [17]:
%sql postgresql://jack@localhost/boutique

In [18]:
%%sql
SELECT * FROM supplier;

id,fk_company_id,contact,phone,job_title
299,194,Johnny Ryall,597-500-569,CEO
157,346,George Clinton,131-002-530,Sales manager


## Exercices


### E201 - Guess who is the CEO of "Johnny's Shoe Store"

Masking the supplier name is clearly not enough to provide anonymity.

**Write a simple SQL query that would reindentify the suppliers based on their job and ther company.**

Company names and job positions are available in many public datasets. A simple search on Linkedin or Google, would give you the names of the top executives of most companies..  

> This is called **Linkability** : the ability to connect multiple records concerning the same data subject.

### E202 - Anonymize the companies  

We need to anonymize the "company" table, too. Even if they don't contain personal information, some fields can be used to **infer** the identity of their employees... 

**Write 2 masking rules for the company table. The first one will replace the "name" field with a fake name. The second will replace the "vat_id" with a random sequence of 10 characters"

> HINT: Go to the [documentation] and look at the [faking functions] and [random functions] !

[documentation]: https://postgresql-anonymizer.readthedocs.io/en/stable/
[faking functions]: https://postgresql-anonymizer.readthedocs.io/en/stable/masking_functions.html#faking
[random functions]: https://postgresql-anonymizer.readthedocs.io/en/stable/masking_functions.html#randomization

Connect as Pierre and check that he cant view the real company info

### E203 - Pseudonymize the company name

Because of dynamic masking, the fake values will be different everytime Pierre tries to read the table.

Pierre would like to have always the same fake values for a given company. **This is called pseudonymization** : generating consistently the same 

**Write a new masking rule over the "vat_id" field by generating 10 random characters using the md5() function.**

**Write a new masking rule over the "name" field by using a [pseudonymizing function].**

[pseudonymizing function]: https://postgresql-anonymizer.readthedocs.io/en/stable/masking_functions.html#pseudonymization



## Solutions

### S201

In [19]:
%sql postgresql://paul@localhost/boutique

In [20]:
%%sql
SELECT s.id, s.contact, s.job_title, c.name  
FROM supplier s 
JOIN company c ON s.fk_company_id = c.id;

id,contact,job_title,name
299,Johnny Ryall,CEO,Johnny's Shoe Store
157,George Clinton,Sales manager,Capitol Records


## S202

In [30]:
%sql postgresql://paul@localhost/boutique

In [31]:
%%capture
%%sql
SECURITY LABEL FOR  anon ON COLUMN company.name
IS 'MASKED WITH FUNCTION anon.fake_company()';

SECURITY LABEL FOR  anon ON COLUMN company.vat_id
IS 'MASKED WITH FUNCTION anon.random_string(10)';

Now connect as Pierre and read the table again:

In [38]:
%sql postgresql://pierre@localhost/boutique

In [36]:
%%sql
SELECT * FROM company;

id,name,vat_id
952,ExoGallery,5V2R2NQEC3
194,Legally Graphic,TH2AWK14JU
346,Home Friend,BWYYZ6HEBE


Pierre will see different "fake data" everytime he reads the table

In [45]:
%%sql
SELECT * FROM company;

id,name,vat_id
952,None,3EVS7T96TC
194,None,DFCW8123KZ
346,Intellipharm,AFMI5427VW


## S203


In [25]:
%sql postgresql://paul@localhost/boutique

In [26]:
%%capture
%%sql
SECURITY LABEL FOR  anon ON COLUMN company.name
IS 'MASKED WITH FUNCTION anon.pseudo_company(id::TEXT)';

Connect as Pierre and read the table multiple times...

In [27]:
%sql postgresql://pierre@localhost/boutique

In [28]:
%%sql
SELECT * FROM company;

id,name,vat_id
952,Landmark Dental,MYJVMMM00M
194,Katakana,0FQOQVSWQ7
346,Tempest Glassworks,B1YFLM6Z6F


In [29]:
%%sql
SELECT * FROM company;

id,name,vat_id
952,Landmark Dental,7I2H3FTG5K
194,Katakana,SWP8B5USSA
346,Tempest Glassworks,QS1FYDRU2O


Now the fake company name is always the same.
